# Homework 1

* Related with the nature of the data, which conditions should meet matrices $A$ and $b$ to be solvable from the least squares perspective.

* Use a least squares for non-linear models adjust the bitcoin price and perform a prediction using the model to determine the future daily price a week ahead, compare with the actual values.

* Use least squares to determine the relationship between the bitcoin price and prices of three comodities namely, oil, silved and gold, during the last year.

* Solve the [point set registration problem](https://en.wikipedia.org/wiki/Point-set_registration). Hint (check: S. Umeyama, Least-Squares Estimation of Transformation Parameters Between Two Point Patterns, IEEE Trans. Pattern Anal. Mach. Intell., vol. 13, no. 4, 1991). Construct a GUI to test. (**Additional points**)

### 1.
El problema de ajuste de datos mediante un modelo se puede resolver mediante la minimización de cuadrados de la siguiente forma:

$$\tilde{x} = \mbox{arg min}_{x} \| Ax - b \|_2^2$$ 
En donde la matriz $A$ "codifica" la naturaleza del modelo, $x$ son los parametros y $b$ son las predicciones.

Expandiendo esta expresion e igualando a cero el gradiente(para hallar el minimo de la función) llegamos a las ecuaciones normales: $$A^T A x = A^T b$$

Ahora bien, ¿Para que $A$ y $b$ tiene solución esta ecuación? 

Si las columnas de A son linearmente independientes, entonces $(A^T A)^{-1}$ existe y la solución se puede calcular mediante $$x = (A^T A)^{-1} A^T b$$

Si las columnas de A **NO** son linearmente independientes, entonces la ecuación se puede solcuionar mediante descomposicion en valores simples.

Por lo tanto, las ecuaciones normales siempre tienen solución y por lo tanto el problema de los minimos cuadrados es solucionable.

### 2. 
Los datos de Bitcoin fueron tomados de [CryptoDataDownload](https://www.cryptodatadownload.com/data/gemini/).

#### Imports

In [1]:
using CSV
using DataFrames
using Plots
using TimeSeries
using LinearAlgebra

Load the Bitcoin dataset

In [ ]:
btc_ds = select(CSV.read("Gemini_BTCUSD_day.csv", DataFrame), Not(["Unix Timestamp", "Symbol"]));
btc_ds.Date = broadcast(d -> Date(d[1:length(d)-9]), btc_ds.Date) # parse dates
first(btc_ds, 5)

In [ ]:
data = (date = btc_ds.Date, Price = btc_ds.Open)
ta = TimeArray(data; timestamp = :date, meta = "btc")
Plots.plot(ta)

Matrix representation of system

In [ ]:
n = 30
btc_prices= btc_ds.Open[1:n];

A = [Array(1:n) ones(n,1)] 
b = Array(btc_prices);

Plot of the objective function $f:\mathbb{R}^2 \to \mathbb{R}$ defined by $f(x) = \| Ax - b \|_2^2$

In [ ]:
function f(x, y)
    X = transpose([x y]) # column vector
    (transpose(A*X-b)*(A*X-b))[1] # 1 element of 1x1 matrix
end
plotlyjs()
surface(-50000:1000:50000,-10000:100:15000, f)

Computing the solution $ x = (A^T A)^{-1} A^T b$

In [ ]:
@time inv(transpose(A)*A)*transpose(A)*b

Moore Penrose pseudoinverse

In [ ]:
@time pinv(A)*b

Solving the normal equations $A^T A x = A^T b$ directly using Julia's backslash operator

In [ ]:
@time transpose(A)*A \ transpose(A)*b

Solving(Approximating) the overdetermined system directly using backslash operator

In [ ]:
@time A \ b

In [ ]:
A = 